In [ ]:
"""
PDF Compression Tool with Image-to-PDF Conversion
Features:
- Upload and reorder images to create PDF
- Compress existing PDFs with customizable settings
- Drag-and-drop interface
- Real-time progress feedback
"""

import gradio as gr
import subprocess
import os
import sys
from pathlib import Path
from typing import List, Tuple, Optional, Dict
import tempfile
import shutil
from PIL import Image
import fitz  # PyMuPDF
import base64
from io import BytesIO

# Check if required packages are installed
try:
    import gradio
except ImportError:
    print("Installing gradio...")
    subprocess.run([sys.executable, "-m", "pip", "install", "gradio>=5.0.0"], check=True)

In [ ]:
# ============================================================================
# COMPRESSION UTILITIES
# ============================================================================

def get_file_size_mb(filepath: str) -> float:
    """Get file size in MB"""
    return os.path.getsize(filepath) / (1024 * 1024)


def compress_pdf_with_settings(
    input_file: str, 
    output_file: str, 
    dpi: int = 72, 
    image_quality: int = 50
) -> Tuple[bool, str]:
    """
    Compress PDF with specific settings using Ghostscript
    
    Args:
        input_file: Input PDF path
        output_file: Output PDF path
        dpi: Image resolution (lower = smaller file)
        image_quality: JPEG quality 0-100 (lower = smaller file)
    
    Returns:
        Tuple of (success: bool, message: str)
    """
    cmd = [
        'gs', '-sDEVICE=pdfwrite', '-dCompatibilityLevel=1.4',
        '-dPDFSETTINGS=/screen', '-dNOPAUSE', '-dQUIET', '-dBATCH',
        '-dDownsampleColorImages=true', '-dDownsampleGrayImages=true', '-dDownsampleMonoImages=true',
        f'-dColorImageResolution={dpi}', f'-dGrayImageResolution={dpi}', f'-dMonoImageResolution={dpi}',
        '-dColorImageDownsampleType=/Bicubic', '-dGrayImageDownsampleType=/Bicubic',
        '-dMonoImageDownsampleType=/Bicubic', f'-dJPEGQ={image_quality}',
        '-dDetectDuplicateImages=true', '-dCompressFonts=true', '-dSubsetFonts=true',
        '-dEmbedAllFonts=true', '-dAutoRotatePages=/None',
        '-dColorConversionStrategy=/LeaveColorUnchanged',
        '-dDoThumbnails=false', '-dCreateJobTicket=false',
        '-dPreserveEPSInfo=false', '-dPreserveOPIComments=false',
        f'-sOutputFile={output_file}', input_file
    ]
    
    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        return True, "Compression successful"
    except subprocess.CalledProcessError as e:
        return False, f"Compression error: {e.stderr}"
    except FileNotFoundError:
        return False, "Ghostscript not found. Please install: sudo apt-get install ghostscript"


def compress_to_target_size(
    input_file: str,
    output_file: str,
    target_size_mb: float = 0.5,
    max_attempts: int = 11
) -> Dict:
    """
    Progressively compress PDF to reach target size
    
    Returns:
        Dict with compression results and statistics
    """
    if not os.path.exists(input_file):
        return {
            "success": False,
            "error": f"Input file not found: {input_file}",
            "initial_size": 0,
            "final_size": 0,
            "reduction_percent": 0
        }
    
    initial_size = get_file_size_mb(input_file)
    
    # Compression configurations (DPI, JPEG Quality, Description)
    configs = [
        (150, 80, "High quality"), (120, 70, "Good quality"),
        (96, 60, "Medium quality"), (72, 50, "Low quality"),
        (50, 40, "Very low quality"), (36, 30, "Minimal quality"),
        (30, 25, "Super compressed"), (24, 20, "Heavy compression"),
        (20, 15, "Maximum compression"), (15, 10, "Extreme compression"),
        (10, 5, "Ultra compression")
    ]
    
    best_config = None
    
    for i, (dpi, quality, desc) in enumerate(configs[:max_attempts]):
        success, msg = compress_pdf_with_settings(input_file, output_file, dpi, quality)
        
        if not success:
            continue
            
        output_size = get_file_size_mb(output_file)
        
        if output_size <= target_size_mb:
            best_config = (dpi, quality, desc)
            break
        
        best_config = (dpi, quality, desc)
    
    if os.path.exists(output_file):
        final_size = get_file_size_mb(output_file)
        reduction = ((initial_size - final_size) / initial_size) * 100 if initial_size > 0 else 0
        
        return {
            "success": final_size <= target_size_mb,
            "initial_size": initial_size,
            "final_size": final_size,
            "reduction_percent": reduction,
            "config_used": best_config,
            "target_reached": final_size <= target_size_mb,
            "output_path": output_file
        }
    
    return {
        "success": False,
        "error": "Failed to compress PDF",
        "initial_size": initial_size,
        "final_size": 0,
        "reduction_percent": 0
    }

In [ ]:
# ============================================================================
# IMAGE TO PDF CONVERSION UTILITIES
# ============================================================================

def convert_images_to_pdf(image_paths: List[str], output_pdf: str) -> Tuple[bool, str]:
    """
    Convert a list of images to a single PDF
    
    Args:
        image_paths: List of image file paths in desired order
        output_pdf: Output PDF path
    
    Returns:
        Tuple of (success: bool, message: str)
    """
    if not image_paths:
        return False, "No images provided"
    
    try:
        # Filter out None values and invalid paths
        valid_images = [p for p in image_paths if p and os.path.exists(p)]
        
        if not valid_images:
            return False, "No valid image files found"
        
        # Create PDF document
        pdf_document = fitz.open()
        
        for img_path in valid_images:
            try:
                # Open image and convert to PDF
                img_doc = fitz.open(img_path)
                pdf_bytes = img_doc.convert_to_pdf()
                img_pdf = fitz.open("pdf", pdf_bytes)
                pdf_document.insert_pdf(img_pdf)
                img_doc.close()
                img_pdf.close()
            except Exception as e:
                # Try with PIL if fitz fails
                try:
                    img = Image.open(img_path)
                    if img.mode == 'RGBA':
                        img = img.convert('RGB')
                    elif img.mode not in ['RGB', 'L']:
                        img = img.convert('RGB')
                    
                    # Save as temporary PDF
                    temp_pdf = tempfile.NamedTemporaryFile(suffix='.pdf', delete=False)
                    img.save(temp_pdf.name, 'PDF', resolution=100.0)
                    temp_pdf.close()
                    
                    # Add to main PDF
                    temp_doc = fitz.open(temp_pdf.name)
                    pdf_document.insert_pdf(temp_doc)
                    temp_doc.close()
                    os.unlink(temp_pdf.name)
                except Exception as e2:
                    return False, f"Failed to process image {os.path.basename(img_path)}: {str(e2)}"
        
        # Save the final PDF
        pdf_document.save(output_pdf)
        pdf_document.close()
        
        return True, f"Successfully created PDF with {len(valid_images)} page(s)"
        
    except Exception as e:
        return False, f"Error creating PDF: {str(e)}"


def get_image_info(image_paths: List[str]) -> str:
    """Get summary information about uploaded images"""
    if not image_paths:
        return "No images uploaded"
    
    valid_images = [p for p in image_paths if p and os.path.exists(p)]
    
    if not valid_images:
        return "No valid images found"
    
    total_size = sum(os.path.getsize(p) for p in valid_images) / (1024 * 1024)
    
    info = f"**📊 Images Info:**\n"
    info += f"- Total images: {len(valid_images)}\n"
    info += f"- Total size: {total_size:.2f} MB\n"
    
    return info

In [ ]:
# ============================================================================
# GRADIO EVENT HANDLERS
# ============================================================================

def process_images_to_pdf_and_compress(
    images: List[str],
    target_size: float,
    dpi: int,
    quality: int,
    auto_compress: bool,
    output_filename: str = "output"
) -> Tuple[Optional[str], str]:
    """
    Process images: convert to PDF and optionally compress
    
    Returns:
        Tuple of (output_file_path, status_message)
    """
    if not images:
        return None, "❌ Please upload at least one image"
    
    # Clean filename
    output_filename = output_filename.strip() or "output"
    # Remove .pdf extension if user added it
    if output_filename.lower().endswith('.pdf'):
        output_filename = output_filename[:-4]
    # Sanitize filename
    output_filename = "".join(c for c in output_filename if c.isalnum() or c in (' ', '-', '_')).strip()
    output_filename = output_filename or "output"
    
    try:
        # Create temporary directory for processing
        temp_dir = tempfile.mkdtemp()
        temp_pdf = os.path.join(temp_dir, f"{output_filename}.pdf")
        
        # Convert images to PDF
        success, msg = convert_images_to_pdf(images, temp_pdf)
        
        if not success:
            shutil.rmtree(temp_dir, ignore_errors=True)
            return None, f"❌ {msg}"
        
        initial_size = get_file_size_mb(temp_pdf)
        
        # Check if compression is needed
        if not auto_compress or initial_size <= target_size:
            result_msg = f"✅ PDF created successfully!\n\n"
            result_msg += f"**📄 PDF Info:**\n"
            result_msg += f"- Filename: {output_filename}.pdf\n"
            result_msg += f"- Pages: {len(images)}\n"
            result_msg += f"- Size: {initial_size:.2f} MB\n"
            
            if not auto_compress:
                result_msg += f"\n*Compression disabled*"
            else:
                result_msg += f"\n*No compression needed (already under {target_size} MB)*"
            
            return temp_pdf, result_msg
        
        # Compress the PDF
        output_pdf = os.path.join(temp_dir, f"{output_filename}_compressed.pdf")
        
        if auto_compress:
            result = compress_to_target_size(temp_pdf, output_pdf, target_size)
        else:
            success, msg = compress_pdf_with_settings(temp_pdf, output_pdf, dpi, quality)
            if success:
                final_size = get_file_size_mb(output_pdf)
                reduction = ((initial_size - final_size) / initial_size) * 100
                result = {
                    "success": True,
                    "initial_size": initial_size,
                    "final_size": final_size,
                    "reduction_percent": reduction,
                    "config_used": (dpi, quality, "Custom")
                }
            else:
                result = {"success": False, "error": msg}
        
        if result.get("success"):
            status = f"✅ PDF created and compressed successfully!\n\n"
            status += f"**📊 Compression Results:**\n"
            status += f"- Filename: {output_filename}_compressed.pdf\n"
            status += f"- Original size: {result['initial_size']:.2f} MB\n"
            status += f"- Compressed size: {result['final_size']:.2f} MB\n"
            status += f"- Reduction: {result['reduction_percent']:.1f}%\n"
            
            if result.get('config_used'):
                dpi_used, qual_used, desc = result['config_used']
                status += f"- Settings: {desc} ({dpi_used} DPI, {qual_used}% quality)\n"
            
            if result.get('target_reached'):
                status += f"\n🎯 Target size ({target_size} MB) reached!"
            else:
                status += f"\n⚠️ Could not reach target size ({target_size} MB) with current settings"
            
            return output_pdf, status
        else:
            return temp_pdf, f"⚠️ PDF created but compression failed: {result.get('error', 'Unknown error')}"
            
    except Exception as e:
        return None, f"❌ Error: {str(e)}"


def compress_existing_pdf(
    pdf_file,
    target_size: float,
    dpi: int,
    quality: int,
    use_auto: bool,
    output_filename: str = "compressed"
) -> Tuple[Optional[str], str]:
    """
    Compress an existing PDF file
    
    Returns:
        Tuple of (output_file_path, status_message)
    """
    if pdf_file is None:
        return None, "❌ Please upload a PDF file"
    
    # Clean filename
    output_filename = output_filename.strip() or "compressed"
    # Remove .pdf extension if user added it
    if output_filename.lower().endswith('.pdf'):
        output_filename = output_filename[:-4]
    # Sanitize filename
    output_filename = "".join(c for c in output_filename if c.isalnum() or c in (' ', '-', '_')).strip()
    output_filename = output_filename or "compressed"
    
    try:
        # Get input file path
        if isinstance(pdf_file, str):
            input_pdf = pdf_file
        else:
            input_pdf = pdf_file.name
        
        if not os.path.exists(input_pdf):
            return None, "❌ PDF file not found"
        
        initial_size = get_file_size_mb(input_pdf)
        
        # Create temporary output file
        temp_dir = tempfile.mkdtemp()
        output_pdf = os.path.join(temp_dir, f"{output_filename}.pdf")
        
        # Compress based on mode
        if use_auto:
            result = compress_to_target_size(input_pdf, output_pdf, target_size)
        else:
            success, msg = compress_pdf_with_settings(input_pdf, output_pdf, dpi, quality)
            if success:
                final_size = get_file_size_mb(output_pdf)
                reduction = ((initial_size - final_size) / initial_size) * 100
                result = {
                    "success": True,
                    "initial_size": initial_size,
                    "final_size": final_size,
                    "reduction_percent": reduction,
                    "config_used": (dpi, quality, "Custom")
                }
            else:
                result = {"success": False, "error": msg}
        
        if result.get("success"):
            status = f"✅ PDF compressed successfully!\n\n"
            status += f"**📊 Compression Results:**\n"
            status += f"- Filename: {output_filename}.pdf\n"
            status += f"- Original size: {result['initial_size']:.2f} MB\n"
            status += f"- Compressed size: {result['final_size']:.2f} MB\n"
            status += f"- Reduction: {result['reduction_percent']:.1f}%\n"
            
            if result.get('config_used'):
                dpi_used, qual_used, desc = result['config_used']
                status += f"- Settings: {desc} ({dpi_used} DPI, {qual_used}% quality)\n"
            
            if use_auto and result.get('target_reached'):
                status += f"\n🎯 Target size ({target_size} MB) reached!"
            elif use_auto:
                status += f"\n⚠️ Could not reach target size ({target_size} MB)"
            
            return output_pdf, status
        else:
            return None, f"❌ Compression failed: {result.get('error', 'Unknown error')}"
            
    except Exception as e:
        return None, f"❌ Error: {str(e)}"


def update_compression_controls(use_auto: bool):
    """Show/hide manual controls based on auto mode"""
    return {
        dpi_slider: gr.update(visible=not use_auto),
        quality_slider: gr.update(visible=not use_auto),
    }

In [ ]:
# ============================================================================
# GRADIO INTERFACE
# ============================================================================

# Custom CSS for professional styling
custom_css = """
.gradio-container {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}

.main-header {
    text-align: center;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 2rem;
    border-radius: 10px;
    margin-bottom: 2rem;
}

.logo-container {
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 15px;
    margin-bottom: 10px;
}

.logo-img {
    width: 60px;
    height: 60px;
    object-fit: contain;
    filter: drop-shadow(0 2px 4px rgba(0,0,0,0.2));
}

.info-box {
    background-color: #f0f7ff;
    border-left: 4px solid #3b82f6;
    padding: 1rem;
    margin: 1rem 0;
    border-radius: 5px;
    color: #1e3a8a;
}

.success-box {
    background-color: #f0fdf4;
    border-left: 4px solid #22c55e;
    padding: 1rem;
    margin: 1rem 0;
    border-radius: 5px;
}

.warning-box {
    background-color: #fffbeb;
    border-left: 4px solid #f59e0b;
    padding: 1rem;
    margin: 1rem 0;
    border-radius: 5px;
}

.compact-slider .wrap {
    gap: 0.5rem !important;
}

.image-item {
    display: flex;
    align-items: center;
    padding: 0.5rem;
    margin: 0.25rem 0;
    background: #f8fafc;
    border-radius: 5px;
    border: 1px solid #e2e8f0;
}

.image-item:hover {
    background: #f1f5f9;
}

.hidden-component {
    display: none !important;
}

footer {
    display: none !important;
}

.pdf-preview-container {
    width: 100%;
    height: 100%;
    border: 1px solid #e2e8f0;
    border-radius: 8px;
    overflow: hidden;
}

.pdf-preview-empty {
    display: flex;
    align-items: center;
    justify-content: center;
    height: 100%;
    color: #94a3b8;
    font-size: 14px;
    background: #f8fafc;
}
"""

# Theme (pass into launch to avoid Gradio 6.0 warning)
app_theme = gr.themes.Soft(
    primary_hue="violet",
    secondary_hue="blue",
)

EMPTY_LIST_HTML = "<p style='color: #64748b; text-align: center; padding: 2rem;'>No images uploaded</p>"

# Logo URL
LOGO_URL = "https://lh3.googleusercontent.com/d/1oqHN_i_fferq7L-5dtnhRY1i1rknwvyk"

# App title
APP_TITLE = "Bủm Xiu PDF Compression Tool"


def get_image_thumbnail(path: str) -> str:
    """Return base64 data URL thumbnail for use in HTML list."""
    try:
        with Image.open(path) as img:
            if img.mode in ("RGBA", "P"):
                img = img.convert("RGB")
            img.thumbnail((80, 80))
            buffered = BytesIO()
            img.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
            return f"data:image/png;base64,{img_str}"
    except Exception:
        return ""


def load_preview_image(path: Optional[str]):
    """Load image for full preview (returned as PIL image)."""
    if not path or not os.path.exists(path):
        return None
    try:
        img = Image.open(path)
        if img.mode in ("RGBA", "P"):
            img = img.convert("RGB")
        max_side = 2500
        if max(img.size) > max_side:
            img.thumbnail((max_side, max_side))
        return img
    except Exception:
        return None


def create_pdf_preview_html(pdf_path: Optional[str], height: int = 420) -> str:
    """Create HTML with iframe to preview PDF."""
    if not pdf_path or not os.path.exists(pdf_path):
        return f'<div class="pdf-preview-empty" style="height: {height}px;">No PDF to preview</div>'
    
    try:
        # Read PDF and encode as base64
        with open(pdf_path, "rb") as f:
            pdf_data = f.read()
        pdf_b64 = base64.b64encode(pdf_data).decode("utf-8")
        
        # Create iframe with embedded PDF
        return f'''
        <div class="pdf-preview-container" style="height: {height}px;">
            <iframe 
                src="data:application/pdf;base64,{pdf_b64}" 
                width="100%" 
                height="100%" 
                style="border: none;"
                type="application/pdf">
                <p>Your browser does not support PDF preview. <a href="data:application/pdf;base64,{pdf_b64}" download>Download PDF</a></p>
            </iframe>
        </div>
        '''
    except Exception as e:
        return f'<div class="pdf-preview-empty" style="height: {height}px;">Error loading PDF preview</div>'


def _images_pipeline(paths, filename, target, dpi, qual, auto):
    """Pipeline for image-to-PDF conversion."""
    out_path, status = process_images_to_pdf_and_compress(paths, target, dpi, qual, auto, filename)
    if out_path:
        preview_html = create_pdf_preview_html(out_path, height=420)
        return out_path, preview_html, status
    return None, '<div class="pdf-preview-empty" style="height: 420px;">No PDF generated</div>', status


def _pdf_pipeline(file, filename, target, dpi, qual, auto):
    """Pipeline for PDF compression."""
    out_path, status = compress_existing_pdf(file, target, dpi, qual, auto, filename)
    if out_path:
        preview_html = create_pdf_preview_html(out_path, height=520)
        return out_path, preview_html, status
    return None, '<div class="pdf-preview-empty" style="height: 520px;">No PDF generated</div>', status


with gr.Blocks(title=APP_TITLE, css=custom_css, theme=app_theme) as demo:

    # Header với logo
    logo_img = f'<img src="{LOGO_URL}" alt="Logo" class="logo-img" onerror="this.style.display=\'none\'">' if LOGO_URL else ""
    gr.Markdown(
        f"""
        <div class="main-header">
            <div class="logo-container">
                {logo_img}
                <h1 style="margin: 0;">{APP_TITLE}</h1>
            </div>
            <p>Convert images to PDF and compress with customizable settings</p>
        </div>
        """,
        elem_classes=["main-header"],
    )

    with gr.Tabs():

        # ====================================================================
        # TAB 1: Images to PDF + Compress
        # ====================================================================
        with gr.Tab("📸 Images to PDF"):
            gr.Markdown(
                """
                <div class="info-box">
                ℹ️ <strong>Click an item to preview the full image.</strong><br/>
                Use ↑↓ to reorder; PDF pages follow the numbered order.
                </div>
                """
            )

            with gr.Row():
                with gr.Column(scale=2):
                    img_upload = gr.File(
                        label="📤 Upload Images",
                        file_count="multiple",
                        file_types=["image"],
                        type="filepath",
                    )

                    img_paths_state = gr.State([])

                    img_list_display = gr.HTML(
                        value=EMPTY_LIST_HTML,
                        label="📋 Image Order",
                    )

                    img_info = gr.Markdown(value="No images uploaded")

                    gr.Markdown("### 📝 Output Settings")
                    img_output_name = gr.Textbox(
                        label="Output PDF Name",
                        value="output",
                        placeholder="Enter filename (without .pdf extension)",
                        info="Name for the generated PDF file",
                    )

                    gr.Markdown("### ⚙️ Compression Settings")

                    with gr.Row():
                        img_auto_compress = gr.Checkbox(
                            label="🎯 Auto-compress to target size",
                            value=True,
                            info="Automatically find best settings",
                        )
                        img_target_size = gr.Slider(
                            minimum=0.1,
                            maximum=10.0,
                            value=0.5,
                            step=0.1,
                            label="Target Size (MB)",
                            elem_classes=["compact-slider"],
                        )

                    with gr.Row():
                        img_dpi = gr.Slider(
                            minimum=10,
                            maximum=300,
                            value=72,
                            step=5,
                            label="DPI (Image Resolution)",
                            visible=False,
                            elem_classes=["compact-slider"],
                        )
                        img_quality = gr.Slider(
                            minimum=5,
                            maximum=100,
                            value=50,
                            step=5,
                            label="JPEG Quality (%)",
                            visible=False,
                            elem_classes=["compact-slider"],
                        )

                    with gr.Row():
                        img_process_btn = gr.Button(
                            "🚀 Create & Compress PDF",
                            variant="primary",
                            size="lg",
                        )
                        img_clear_btn = gr.Button(
                            "🗑️ Clear",
                            variant="secondary",
                        )

                with gr.Column(scale=1):
                    img_preview = gr.Image(
                        label="🖼️ Image Preview",
                        value=None,
                        height=280,
                    )

                    img_pdf_preview = gr.HTML(
                        value='<div class="pdf-preview-empty" style="height: 420px;">No PDF to preview</div>',
                        label="📄 PDF Preview (scroll)",
                    )

                    img_download_btn = gr.DownloadButton(
                        label="⬇️ Download PDF",
                        value=None,
                        interactive=True,
                    )

                    img_status = gr.Markdown(
                        value="Upload images to get started",
                        label="Status",
                    )

            def render_image_list(paths: List[str]) -> str:
                if not paths:
                    return EMPTY_LIST_HTML

                html = "<div style='max-height: 500px; overflow-y: auto; padding-right: 5px;'>"
                for idx, path in enumerate(paths):
                    filename = os.path.basename(path)
                    disabled_up = idx == 0
                    disabled_down = idx == len(paths) - 1

                    thumb_src = get_image_thumbnail(path)
                    img_tag = (
                        f'<img src="{thumb_src}" '
                        'style="width: 60px; height: 60px; object-fit: cover; border-radius: 4px; '
                        'margin: 0 10px; border: 1px solid #cbd5e1;" />'
                        if thumb_src
                        else ""
                    )

                    # Styles cho các nút với màu sắc rõ ràng
                    btn_up_style = (
                        "padding: 8px 14px; border: none; border-radius: 5px; cursor: pointer; "
                        "font-size: 14px; font-weight: bold; transition: all 0.2s; min-width: 40px; "
                        f"background: {'#e2e8f0' if disabled_up else '#3b82f6'}; "
                        f"color: {'#94a3b8' if disabled_up else 'white'}; "
                        f"cursor: {'not-allowed' if disabled_up else 'pointer'}; "
                        "box-shadow: 0 2px 4px rgba(0,0,0,0.1);"
                    )
                    
                    btn_down_style = (
                        "padding: 8px 14px; border: none; border-radius: 5px; cursor: pointer; "
                        "font-size: 14px; font-weight: bold; transition: all 0.2s; min-width: 40px; "
                        f"background: {'#e2e8f0' if disabled_down else '#3b82f6'}; "
                        f"color: {'#94a3b8' if disabled_down else 'white'}; "
                        f"cursor: {'not-allowed' if disabled_down else 'pointer'}; "
                        "box-shadow: 0 2px 4px rgba(0,0,0,0.1);"
                    )
                    
                    btn_remove_style = (
                        "padding: 8px 14px; background: #ef4444; color: white; border: none; "
                        "border-radius: 5px; cursor: pointer; font-size: 14px; font-weight: bold; "
                        "transition: all 0.2s; min-width: 40px; box-shadow: 0 2px 4px rgba(239,68,68,0.2);"
                    )

                    # Hover effects
                    btn_up_hover = "" if disabled_up else (
                        "onmouseover=\"this.style.background='#2563eb'; this.style.transform='translateY(-1px)';\" "
                        "onmouseout=\"this.style.background='#3b82f6'; this.style.transform='translateY(0)';\""
                    )
                    
                    btn_down_hover = "" if disabled_down else (
                        "onmouseover=\"this.style.background='#2563eb'; this.style.transform='translateY(-1px)';\" "
                        "onmouseout=\"this.style.background='#3b82f6'; this.style.transform='translateY(0)';\""
                    )
                    
                    btn_remove_hover = (
                        "onmouseover=\"this.style.background='#dc2626'; this.style.transform='translateY(-1px)';\" "
                        "onmouseout=\"this.style.background='#ef4444'; this.style.transform='translateY(0)';\""
                    )

                    html += f"""
                    <div class='image-item'
                         style='display: flex; align-items: center; gap: 10px; padding: 8px; margin: 6px 0; 
                                background: #f8fafc; border-radius: 8px; border: 1px solid #e2e8f0; 
                                box-shadow: 0 1px 2px rgba(0,0,0,0.05); cursor: pointer;'
                         onclick='document.getElementById("select_{idx}").click()'
                         onmouseover="this.style.background='#f1f5f9';"
                         onmouseout="this.style.background='#f8fafc';">
                        <span style='font-weight: bold; color: #3b82f6; min-width: 30px; text-align: center;'>#{idx + 1}</span>
                        {img_tag}
                        <span style='flex: 1; color: #334155; font-size: 14px; font-weight: 500; 
                                     overflow: hidden; text-overflow: ellipsis; white-space: nowrap;'>{filename}</span>
                        <div style='display: flex; gap: 6px;'>
                            <button onclick='(function(e){{e.stopPropagation();document.getElementById("move_up_{idx}").click();}})(event)'
                                    style='{btn_up_style}'
                                    {btn_up_hover}
                                    {'disabled' if disabled_up else ''}
                                    title='Move Up'>↑</button>
                            <button onclick='(function(e){{e.stopPropagation();document.getElementById("move_down_{idx}").click();}})(event)'
                                    style='{btn_down_style}'
                                    {btn_down_hover}
                                    {'disabled' if disabled_down else ''}
                                    title='Move Down'>↓</button>
                            <button onclick='(function(e){{e.stopPropagation();document.getElementById("remove_{idx}").click();}})(event)'
                                    style='{btn_remove_style}'
                                    {btn_remove_hover}
                                    title='Remove'>✕</button>
                        </div>
                    </div>
                    """
                html += "</div>"
                return html

            def update_image_list(files):
                if files:
                    paths = [f.name if hasattr(f, "name") else f for f in files]
                    preview = load_preview_image(paths[0]) if paths else None
                    return paths, render_image_list(paths), get_image_info(paths), preview
                return [], EMPTY_LIST_HTML, "No images uploaded", None

            def move_image_up(paths, idx):
                if paths and 0 < idx < len(paths):
                    new_paths = list(paths)
                    new_paths[idx], new_paths[idx - 1] = new_paths[idx - 1], new_paths[idx]
                    preview = load_preview_image(new_paths[0]) if new_paths else None
                    return new_paths, render_image_list(new_paths), get_image_info(new_paths), preview
                preview = load_preview_image(paths[0]) if paths else None
                return paths, render_image_list(paths), get_image_info(paths), preview

            def move_image_down(paths, idx):
                if paths and 0 <= idx < len(paths) - 1:
                    new_paths = list(paths)
                    new_paths[idx], new_paths[idx + 1] = new_paths[idx + 1], new_paths[idx]
                    preview = load_preview_image(new_paths[0]) if new_paths else None
                    return new_paths, render_image_list(new_paths), get_image_info(new_paths), preview
                preview = load_preview_image(paths[0]) if paths else None
                return paths, render_image_list(paths), get_image_info(paths), preview

            def remove_image(paths, idx):
                if paths and 0 <= idx < len(paths):
                    new_paths = list(paths)
                    new_paths.pop(idx)
                    preview = load_preview_image(new_paths[0]) if new_paths else None
                    return new_paths, render_image_list(new_paths), get_image_info(new_paths), preview
                preview = load_preview_image(paths[0]) if paths else None
                return paths, render_image_list(paths), get_image_info(paths), preview

            def select_image(paths, idx):
                if paths and 0 <= idx < len(paths):
                    return load_preview_image(paths[idx])
                return None

            with gr.Row():
                for i in range(50):
                    btn_up = gr.Button("", elem_id=f"move_up_{i}", elem_classes=["hidden-component"])
                    btn_down = gr.Button("", elem_id=f"move_down_{i}", elem_classes=["hidden-component"])
                    btn_remove = gr.Button("", elem_id=f"remove_{i}", elem_classes=["hidden-component"])
                    btn_select = gr.Button("", elem_id=f"select_{i}", elem_classes=["hidden-component"])

                    btn_up.click(
                        fn=lambda paths, index=i: move_image_up(paths, index),
                        inputs=[img_paths_state],
                        outputs=[img_paths_state, img_list_display, img_info, img_preview],
                    )
                    btn_down.click(
                        fn=lambda paths, index=i: move_image_down(paths, index),
                        inputs=[img_paths_state],
                        outputs=[img_paths_state, img_list_display, img_info, img_preview],
                    )
                    btn_remove.click(
                        fn=lambda paths, index=i: remove_image(paths, index),
                        inputs=[img_paths_state],
                        outputs=[img_paths_state, img_list_display, img_info, img_preview],
                    )
                    btn_select.click(
                        fn=lambda paths, index=i: select_image(paths, index),
                        inputs=[img_paths_state],
                        outputs=[img_preview],
                    )

            img_upload.change(
                fn=update_image_list,
                inputs=[img_upload],
                outputs=[img_paths_state, img_list_display, img_info, img_preview],
            )

            img_auto_compress.change(
                fn=lambda x: (gr.update(visible=not x), gr.update(visible=not x)),
                inputs=[img_auto_compress],
                outputs=[img_dpi, img_quality],
            )

            img_process_btn.click(
                fn=_images_pipeline,
                inputs=[
                    img_paths_state,
                    img_output_name,
                    img_target_size,
                    img_dpi,
                    img_quality,
                    img_auto_compress,
                ],
                outputs=[img_download_btn, img_pdf_preview, img_status],
            )

            img_clear_btn.click(
                fn=lambda: (
                    None,
                    [],
                    EMPTY_LIST_HTML,
                    "No images uploaded",
                    None,
                    '<div class="pdf-preview-empty" style="height: 420px;">No PDF to preview</div>',
                    None,
                    "output",
                    "Upload images to get started",
                ),
                outputs=[
                    img_upload,
                    img_paths_state,
                    img_list_display,
                    img_info,
                    img_preview,
                    img_pdf_preview,
                    img_download_btn,
                    img_output_name,
                    img_status,
                ],
            )

        # ====================================================================
        # TAB 2: Compress Existing PDF
        # ====================================================================
        with gr.Tab("📎 Compress PDF"):
            gr.Markdown(
                """
                <div class="info-box">
                ℹ️ <strong>Upload an existing PDF file to compress it.</strong>
                Choose between automatic compression or manual settings.
                </div>
                """
            )

            with gr.Row():
                with gr.Column(scale=2):
                    pdf_upload = gr.File(
                        label="📤 Upload PDF",
                        file_types=[".pdf"],
                        type="filepath",
                    )

                    pdf_info = gr.Markdown(value="No PDF uploaded")

                    gr.Markdown("### 📝 Output Settings")
                    pdf_output_name = gr.Textbox(
                        label="Output PDF Name",
                        value="compressed",
                        placeholder="Enter filename (without .pdf extension)",
                        info="Name for the compressed PDF file",
                    )

                    gr.Markdown("### ⚙️ Compression Settings")

                    with gr.Row():
                        pdf_auto_compress = gr.Checkbox(
                            label="🎯 Auto-compress to target size",
                            value=True,
                            info="Automatically find best settings",
                        )
                        pdf_target_size = gr.Slider(
                            minimum=0.1,
                            maximum=10.0,
                            value=0.5,
                            step=0.1,
                            label="Target Size (MB)",
                            elem_classes=["compact-slider"],
                        )

                    with gr.Row():
                        pdf_dpi = gr.Slider(
                            minimum=10,
                            maximum=300,
                            value=72,
                            step=5,
                            label="DPI (Image Resolution)",
                            visible=False,
                            elem_classes=["compact-slider"],
                        )
                        pdf_quality = gr.Slider(
                            minimum=5,
                            maximum=100,
                            value=50,
                            step=5,
                            label="JPEG Quality (%)",
                            visible=False,
                            elem_classes=["compact-slider"],
                        )

                    gr.Markdown("### 📋 Quick Presets")
                    with gr.Row():
                        preset_high = gr.Button("🌟 High Quality", size="sm")
                        preset_medium = gr.Button("⚡ Medium", size="sm")
                        preset_low = gr.Button("💾 Low Size", size="sm")

                    with gr.Row():
                        pdf_compress_btn = gr.Button(
                            "🚀 Compress PDF",
                            variant="primary",
                            size="lg",
                        )
                        pdf_clear_btn = gr.Button(
                            "🗑️ Clear",
                            variant="secondary",
                        )

                with gr.Column(scale=1):
                    pdf_preview = gr.HTML(
                        value='<div class="pdf-preview-empty" style="height: 520px;">No PDF to preview</div>',
                        label="📄 PDF Preview (scroll)",
                    )

                    pdf_download_btn = gr.DownloadButton(
                        label="⬇️ Download PDF",
                        value=None,
                        interactive=True,
                    )

                    pdf_status = gr.Markdown(
                        value="Upload a PDF to get started",
                        label="Status",
                    )

            def update_pdf_info(file):
                if file:
                    try:
                        file_path = file.name if hasattr(file, "name") else file
                        if isinstance(file_path, str) and os.path.exists(file_path):
                            size = get_file_size_mb(file_path)
                            return f"**📊 PDF Info:**\n- Size: {size:.2f} MB"
                    except Exception:
                        pass
                return "No PDF uploaded"

            pdf_upload.change(
                fn=update_pdf_info,
                inputs=[pdf_upload],
                outputs=[pdf_info],
            )

            pdf_auto_compress.change(
                fn=lambda x: (gr.update(visible=not x), gr.update(visible=not x)),
                inputs=[pdf_auto_compress],
                outputs=[pdf_dpi, pdf_quality],
            )

            preset_high.click(
                fn=lambda: (False, 150, 80),
                outputs=[pdf_auto_compress, pdf_dpi, pdf_quality],
            ).then(
                fn=lambda: (gr.update(visible=True), gr.update(visible=True)),
                outputs=[pdf_dpi, pdf_quality],
            )

            preset_medium.click(
                fn=lambda: (False, 96, 60),
                outputs=[pdf_auto_compress, pdf_dpi, pdf_quality],
            ).then(
                fn=lambda: (gr.update(visible=True), gr.update(visible=True)),
                outputs=[pdf_dpi, pdf_quality],
            )

            preset_low.click(
                fn=lambda: (False, 50, 30),
                outputs=[pdf_auto_compress, pdf_dpi, pdf_quality],
            ).then(
                fn=lambda: (gr.update(visible=True), gr.update(visible=True)),
                outputs=[pdf_dpi, pdf_quality],
            )

            pdf_compress_btn.click(
                fn=_pdf_pipeline,
                inputs=[
                    pdf_upload,
                    pdf_output_name,
                    pdf_target_size,
                    pdf_dpi,
                    pdf_quality,
                    pdf_auto_compress,
                ],
                outputs=[pdf_download_btn, pdf_preview, pdf_status],
            )

            pdf_clear_btn.click(
                fn=lambda: (
                    None, 
                    "No PDF uploaded", 
                    "compressed", 
                    '<div class="pdf-preview-empty" style="height: 520px;">No PDF to preview</div>', 
                    None, 
                    "Upload a PDF to get started"
                ),
                outputs=[pdf_upload, pdf_info, pdf_output_name, pdf_preview, pdf_download_btn, pdf_status],
            )

    gr.Markdown(
        """
        ---
        💡 **Tips:**
        - Click an image item to preview the full image
        - Use ↑↓ buttons to reorder images in the list
        - PDF Preview supports scrolling (embedded in iframe)
        - Lower DPI and quality = smaller file size but lower visual quality
        - Auto-compress mode progressively tries different settings to reach target size
        - Supported image formats: PNG, JPG, JPEG, BMP, TIFF, WEBP
        - Requires Ghostscript: `sudo apt-get install ghostscript` (Linux) or `brew install ghostscript` (Mac)
        """
    )

In [ ]:
# ============================================================================
# LAUNCH APPLICATION
# ============================================================================

# Launch the Gradio app
# - allow downloads from tempfile directory (/tmp)
# - pass theme/css here (Gradio 6.0+ prefers this)
demo.launch(
    server_name="0.0.0.0",
    share=True,
    show_error=True,
    quiet=False,
    allowed_paths=[tempfile.gettempdir()],
    theme=app_theme,
    css=custom_css,
)